In [33]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

assert OPENAI_API_KEY, "OPENAI_API_KEY not set"
assert LANGSMITH_API_KEY, "LANGSMITH_API_KEY not set"
assert LANGSMITH_PROJECT, "LANGSMITH_PROJECT not set"
assert TAVILY_API_KEY, "TAVILY_API_KEY not set"

print("All keys loaded.")

All keys loaded.


In [10]:
from langchain_community.document_loaders import PyPDFLoader, UnstructuredExcelLoader
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langsmith import traceable

In [92]:
from langchain.tools import tool

In [37]:
from langchain_tavily import TavilySearch

In [34]:
from langchain.agents import initialize_agent, Tool
from langchain.tools.tavily_search import TavilySearchResults

### Document loading

In [11]:
def load_pdfs(pdf_paths):
    all_pages = []
    for pdf_path in pdf_paths:
        try:
            loader = PyPDFLoader(pdf_path)
            pages = loader.load()
            all_pages.extend(pages)
        except Exception as e:
            print(f"Failed to load PDF ({pdf_path}): {e}")
    print(f"Loaded {len(all_pages)} pages from PDFs.")
    return all_pages

# Test (Replace with your actual file name(s))
PDF_PATHS = ["general_faqs.pdf", "general_faqs_2.pdf"]
pdf_docs = load_pdfs(PDF_PATHS)
pdf_docs[:2]  # See the first two loaded pages

Loaded 9 pages from PDFs.


[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Copy of Health Insurance – Frequently Asked Questions', 'source': 'general_faqs.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Health\n \nInsurance\n \n–\n \nFrequently\n \nAsked\n \nQuestions\n \nQ:  How  do  I  add  a  dependent  to  my  health  insurance  plan?  A:  You  can  add  a  dependent,  such  as  a  spouse  or  child  by  ﬁlling  out  this  form  .  the  link  to  ﬁll  the  form  is  https://docs.google.com/forms/d/e/1FAIpQLSd6RuBtN_mBxH2X3RJdPKdBMS0_YBpqgvFiod\nHBYWLeOG40hg/viewform?usp=sf_link\n \nAdding  new  dependents  usually  takes  up  to  5  days  from  the  form  submission  date.  \nQ:  Can  I  add  my  parents  or  siblings  to  the  plan?  \nA:\n \nOnly\n \nspouses\n \nand\n \nchildren\n \nare\n \neligible\n \nto\n \nbe\n \nadded\n \non\n \nyour\n \nhealth\n \ninsurance\n \nplan.\n \n \nQ:  Does  the  plan  include  gym  

In [16]:
def load_xlsx_langchain(xlsx_paths):
    all_docs = []
    for xlsx_path in xlsx_paths:
        try:
            loader = UnstructuredExcelLoader(xlsx_path, mode="elements")
            docs = loader.load()
            print(f"Loaded {len(docs)} docs from {xlsx_path}")
            all_docs.extend(docs)
        except Exception as e:
            print(f"Failed to load XLSX ({xlsx_path}): {e}")
    print(f"Total loaded from XLSX files: {len(all_docs)}")
    return all_docs

# Test loading Excel files
XLSX_PATHS = ["axa_may_2025_provider_list.xlsx", "customised_plan_and_benefits_2025.xlsx"]
xlsx_docs = load_xlsx_langchain(XLSX_PATHS)
xlsx_docs[:2]

Loaded 92 docs from axa_may_2025_provider_list.xlsx
Loaded 25 docs from customised_plan_and_benefits_2025.xlsx
Total loaded from XLSX files: 117


[Document(metadata={'source': 'axa_may_2025_provider_list.xlsx', 'filename': 'axa_may_2025_provider_list.xlsx', 'last_modified': '2025-07-28T10:12:15', 'page_name': 'General', 'page_number': 1, 'text_as_html': '<table><tr><td>Code</td><td/><td>Address</td><td>State</td><td>City</td><td>Hospital Class</td><td>ServiceType</td><td>EMAIL ADDRESS</td><td>PHONE NUMBER</td></tr><tr><td/><td>HOSPITALS (GENERAL SERVICES)</td><td/><td/><td/><td/><td/><td/><td/></tr><tr><td/><td>LAGOS (ISLAND)</td><td/><td/><td/><td/><td/><td/><td/></tr><tr><td>2340409</td><td>BARNES</td><td>PLOT 34B CHIEF ABIODUN YUSUFU ONIRU ROAD</td><td>LAGOS</td><td>VICTORIA ISLAND</td><td>BRONZE</td><td>GENERAL</td><td>barneshospital1.ng@gmail.com</td><td>08129915303</td></tr><tr><td>2341261</td><td>GROVER MEDICALS LIFESTYLE CLINIC</td><td>81A YOUNIS BASHORUN STREET, OFF AJOSE ADEOGUN, VI</td><td>LAGOS</td><td>VICTORIA ISLAND</td><td>BRONZE</td><td>GENERAL</td><td>info@groverlifestyleclinic.com</td><td>09022012109</td></tr><

In [18]:
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
pdf_chunks = pdf_splitter.split_documents(pdf_docs)
print(f"PDF split into {len(pdf_chunks)} chunks.")
pdf_chunks[:12]


PDF split into 24 chunks.


[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Copy of Health Insurance – Frequently Asked Questions', 'source': 'general_faqs.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Health\n \nInsurance\n \n–\n \nFrequently\n \nAsked\n \nQuestions\n \nQ:  How  do  I  add  a  dependent  to  my  health  insurance  plan?  A:  You  can  add  a  dependent,  such  as  a  spouse  or  child  by  ﬁlling  out  this  form  .  the  link  to  ﬁll  the  form  is  https://docs.google.com/forms/d/e/1FAIpQLSd6RuBtN_mBxH2X3RJdPKdBMS0_YBpqgvFiod\nHBYWLeOG40hg/viewform?usp=sf_link\n \nAdding  new  dependents  usually  takes  up  to  5  days  from  the  form  submission  date.  \nQ:  Can  I  add  my  parents  or  siblings  to  the  plan?  \nA:\n \nOnly\n \nspouses\n \nand\n \nchildren\n \nare\n \neligible\n \nto\n \nbe\n \nadded\n \non\n \nyour\n \nhealth\n \ninsurance\n \nplan.\n \n \nQ:  Does  the  plan  include  gym  

In [19]:
# xlsx_docs are already one row per chunk/document
all_docs_for_embedding = pdf_chunks + xlsx_docs
print(f"Total documents for embedding: {len(all_docs_for_embedding)}")

Total documents for embedding: 141


In [125]:
from pprint import pprint

In [126]:
print(f"Total docs: {len(all_docs_for_embedding)}")

# Look at first 3 docs' metadata
for i, doc in enumerate(all_docs_for_embedding[:3]):
    print(f"\n--- Document {i+1} ---")
    pprint(doc.metadata)

Total docs: 141

--- Document 1 ---
{'creationdate': '',
 'creator': 'PyPDF',
 'page': 0,
 'page_label': '1',
 'producer': 'Skia/PDF m140 Google Docs Renderer',
 'source': 'general_faqs.pdf',
 'title': 'Copy of Health Insurance – Frequently Asked Questions',
 'total_pages': 1}

--- Document 2 ---
{'creationdate': '',
 'creator': 'PyPDF',
 'page': 0,
 'page_label': '1',
 'producer': 'Skia/PDF m140 Google Docs Renderer',
 'source': 'general_faqs.pdf',
 'title': 'Copy of Health Insurance – Frequently Asked Questions',
 'total_pages': 1}

--- Document 3 ---
{'creationdate': '',
 'creator': 'PyPDF',
 'page': 0,
 'page_label': '1',
 'producer': 'Skia/PDF m140 Google Docs Renderer',
 'source': 'general_faqs.pdf',
 'title': 'Copy of Health Insurance – Frequently Asked Questions',
 'total_pages': 1}


In [127]:
all_keys = set()
for doc in all_docs_for_embedding:
    all_keys.update(doc.metadata.keys())

print("\nUnique metadata keys found:")
print(sorted(all_keys))


Unique metadata keys found:
['category', 'creationdate', 'creator', 'element_id', 'filename', 'filetype', 'last_modified', 'page', 'page_label', 'page_name', 'page_number', 'parent_id', 'producer', 'source', 'text_as_html', 'title', 'total_pages']


In [21]:
for idx, doc in enumerate(all_docs_for_embedding):
    print(idx, type(doc))

0 <class 'langchain_core.documents.base.Document'>
1 <class 'langchain_core.documents.base.Document'>
2 <class 'langchain_core.documents.base.Document'>
3 <class 'langchain_core.documents.base.Document'>
4 <class 'langchain_core.documents.base.Document'>
5 <class 'langchain_core.documents.base.Document'>
6 <class 'langchain_core.documents.base.Document'>
7 <class 'langchain_core.documents.base.Document'>
8 <class 'langchain_core.documents.base.Document'>
9 <class 'langchain_core.documents.base.Document'>
10 <class 'langchain_core.documents.base.Document'>
11 <class 'langchain_core.documents.base.Document'>
12 <class 'langchain_core.documents.base.Document'>
13 <class 'langchain_core.documents.base.Document'>
14 <class 'langchain_core.documents.base.Document'>
15 <class 'langchain_core.documents.base.Document'>
16 <class 'langchain_core.documents.base.Document'>
17 <class 'langchain_core.documents.base.Document'>
18 <class 'langchain_core.documents.base.Document'>
19 <class 'langchain_c

### Creating splits

In [23]:
from langchain_core.documents import Document

def prepare_docs_for_embedding(splits):
    clean_splits = []
    for idx, s in enumerate(splits):
        if isinstance(s, Document):
            s.metadata = filter_complex_metadata([s])[0].metadata
            if s.page_content and len(s.page_content.strip()) > 20:
                clean_splits.append(s)
        else:
            print(f"Warning: splits[{idx}] is not a Document (is {type(s)}), skipping. Value: {repr(s)[:80]}")
    print(f"{len(clean_splits)} splits ready for embedding.")
    return clean_splits

# Now run this
clean_splits = prepare_docs_for_embedding(all_docs_for_embedding)


129 splits ready for embedding.


### Embedding docs

In [24]:
def embed_docs(clean_splits, persist_directory="chroma_db"):
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    vectorstore = Chroma.from_documents(clean_splits, embeddings, persist_directory=persist_directory)
    print("Vector store created and persisted.")
    return vectorstore

vectorstore = embed_docs(clean_splits)

Vector store created and persisted.


### Setting up QA chain and building search agent

In [77]:
@traceable
def setup_qa_chain(vectorstore):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    prompt = ChatPromptTemplate.from_template("""
        You are a helpful, accurate, and professional QnA assistant designed to support employees of a Nigerian company with health insurance-related and general HR questions.
        
        Your goal is to provide clear, concise, and context-aware answers based on internal knowledge sources and public information, when required.
        
        ### Audience & Localization
        You are interacting with employees based in Nigeria. You must interpret terms, slang, and geographic references according to local Nigerian usage.
        
        Here are important localization rules to follow:
        - "The Island" refers to the Lagos Island region, including Lekki, Victoria Island (VI), Ikoyi, Ajah, and Oniru.
        - "The Mainland" refers to areas like Yaba, Ikeja, Surulere, Maryland, and Gbagada.
        - "My plan" typically refers to the employee’s active health insurance plan.
        - "HMO" refers to the employee’s health maintenance organization.
        - Always default to Nigerian interpretations unless a user explicitly specifies an international location or context.
        
        If a user uses ambiguous terms, assume they are referring to the Nigerian context. If needed, ask clarifying questions using Nigerian-specific options (e.g., “Do you mean Lagos Island like Lekki or VI?” instead of “Long Island or Bali?”).
        
        ### Knowledge Bases
        You have access to internal documents and are expected to reference them when answering questions:
        
        - `customised_plan_and_benefits_2025.xlsx`: Health plan tiers, benefit details, and coverage limits (e.g., dental, optical, maternity).
        - `axa_may_2025_provider_list.xlsx`: List of in-network hospitals, clinics, their locations, and contact details.
        - Internal PDFs: HR policies, onboarding information, and general employee guides.
        
        Always reference the appropriate document when formulating answers. You may summarize, but do not invent facts that are not present in the source.
        
        ### Web Search
        When a user asks for a recommendation based on public sentiment (e.g., “which hospital on the Island is best for dental care and why?”), feel free to perform a web search to gather recent information like ratings, reviews, or operational status. Return a summary and include the reasoning for your recommendation.
        
        ### Data Privacy
        You must not disclose any employee's insurance details unless you are confident the request is coming from that specific employee.
        To verify a user's identity, only use the provided enrollee ID or authenticated metadata passed to you from the system. Never reveal one employee’s insurance information to another.
        If a user tries to access someone else's information, politely deny the request and explain that insurance profiles are confidential.
        
        ### Prompt Format
        Context: {context}
        Question: {question}

    """)
    llm = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=OPENAI_API_KEY)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        return_source_documents=False,
        chain_type_kwargs={"prompt": prompt},
    )
    return qa_chain

qa_chain = setup_qa_chain(vectorstore)

In [78]:
def answer_question(qa_chain, question):
    try:
        response = qa_chain(question)
        answer = response['result']
        sources = response.get('source_documents', [])
        print(f"Answer: {answer}")
        if sources:
            print("Sources:")
            for doc in sources:
                meta = getattr(doc, 'metadata', {})
                print(f"- {meta.get('source', 'unknown source')} (sheet: {meta.get('sheet', 'N/A')})")
        return answer
    except Exception as e:
        print(f"Error answering question: {e}")
        return "An error occurred while answering the question."

In [107]:
# Example query:
answer_question(qa_chain, "i have a platinum plan on my hmo. do i have access to any spa?")

Answer: Yes, with a Platinum plan on your HMO, you have access to spa services. Specifically, you can access spas on the AXA Mansard Network with a limit of ₦15,000 for either facials or massages.


'Yes, with a Platinum plan on your HMO, you have access to spa services. Specifically, you can access spas on the AXA Mansard Network with a limit of ₦15,000 for either facials or massages.'

In [115]:
# --- Web Search Tool ---
# search = TavilySearch()
# web_tool = Tool(
#     name="search_web",
#     func=search.run,
#     description="Use for up-to-date or public web information (hospital reviews, top lists, latest news, etc)."
# )
search = TavilySearch()
def search_web(query: str) -> str:
    # Add localization hint so results stay Nigeria/Lagos-focused
    prefix = ("Nigerian context. When users say 'the Island', interpret as Lagos Island axis "
              "(Lekki, VI, Ikoyi, Ajah, Oniru). ")
    return search.run(prefix + query)

web_tool = Tool(
    name="search_web",
    func=search_web,
    description="Search the public web for up-to-date info: hospital reviews, rankings, addresses, recent news."
)

In [96]:
@tool
def rag_tool_func(query: str) -> str:
    """
    Use this tool to answer questions about insurance benefits, provider lists, or HR policies using company knowledge base.
    """
    response = qa_chain.invoke({"query": query})
    return response['result']

In [97]:
rag_tool = Tool(
    name="search_internal_docs",
    func=rag_tool_func,
    description="Use for questions answerable with internal company docs (insurance coverage, provider lists, internal FAQs, etc)."
)

In [108]:
nigerian_system_prompt = """
You are a helpful, accurate, and professional QnA assistant supporting employees of a Nigerian fintech company with health insurance, HR, and provider-related questions.

Localization rules:
- 'The Island' means the Lagos Island axis (Lekki, VI, Ikoyi, Ajah, Oniru).
- 'The Mainland' means places like Yaba, Ikeja, Surulere, Maryland.
- Default all context to Nigerian usage and culture.

When users ask for hospital or clinic recommendations, interpret requests in the Nigerian context, using recent data where possible. Never reveal private information about another employee.

Always explain the reasoning for your recommendations. If you don't have enough information, ask clarifying questions using Nigerian examples.
"""

In [112]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(nigerian_system_prompt),
    MessagesPlaceholder("chat_history"),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

In [119]:
# --- Combine both tools ---
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)
tools = [rag_tool, web_tool]
# agent = initialize_agent(
#     tools,
#     llm,
#     agent="zero-shot-react-description",
#     verbose=True,  #shows agent's thought process
#     handle_parsing_errors=True
# )
agent = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [105]:
print(agent.run("recommend the best eye clinic on victoria island for an employee on a gold plan and give me the reviews. tell me why it's the best and add links to pictures of the facility"))



> Entering new AgentExecutor chain...
To recommend the best eye clinic on Victoria Island for an employee on a gold plan, I need to check the internal company documents to see which clinics are covered under the gold plan. Then, I will search the web for reviews and images of the facility.

Action: search_internal_docs
Action Input: "eye clinics on Victoria Island covered under gold plan"
Observation: Based on the information from the `axa_may_2025_provider_list.xlsx`, the following eye clinics on Victoria Island are covered under the Gold plan:

1. **The Eye Doctor**
   - Address: Plot 241B Muri Okunola Street, Victoria Island, Lagos

2. **Swift Eyecare Limited**
   - Address: Plot 1205, Amodu Ojikutu Street, Off Bishop Oluwole Street, Victoria Island, Lagos

These clinics are classified under the Gold plan for optical services. If you need further assistance or details about your specific plan, please let me know!
Thought:I have identified two eye clinics on Victoria Island that ar

In [136]:
result = agent_executor.invoke({
    "input": "recommend the best general hospital in abuja to go to (for a gold plan user) based on public reviews. Also provide the links to their websites",
    "chat_history": []  # or your running history of (AI/User) messages if you keep one
})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_web` with `best general hospital in Abuja reviews 2023`


{'query': "Nigerian context. When users say 'the Island', interpret as Lagos Island axis (Lekki, VI, Ikoyi, Ajah, Oniru). best general hospital in Abuja reviews 2023", 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.limihospital.org/top-5-abuja-best-hospitals/', 'title': 'Top 5 Abuja Best Hospitals', 'content': 'Top 5 Abuja Best Hospitals · 1. Limi Hospital · 2. Nizamiye Hospital (Life Camp) · 3. Kelina Hospital (Gwarimpa) · 4. Maitama General Hospital', 'score': 0.79196626, 'raw_content': None}, {'url': 'https://resuscitatehospital.org/', 'title': 'Resuscitate Hospital: Best Hospital in Abuja, Nigeria', 'content': 'Resuscitate Hospital is the best and affordable hospital in Abuja, Nigeria with many years of experience in proving healthcare services.', 'score': 0.6186197, 'raw_content': None}, {'url': 'https://www.youtube.com/w

In [130]:
from langchain_mongodb import MongoDBAtlasVectorSearch

vectorstore = MongoDBAtlasVectorSearch.from_existing_index(
    embedding=embeddings,
    collection=collection,
    index_name="hmo_vector_index",
    text_key="text",
    embedding_key="embedding",
    metadata_key="metadata"
)

print("Number of embedded docs:", collection.count_documents({}))

AttributeError: type object 'MongoDBAtlasVectorSearch' has no attribute 'from_existing_index'

In [129]:
pip install langchain_mongodb

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_mongodb-0.6.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached lark-1.2.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached langchain_mongodb-0.6.2-py3-none-any.whl (59 kB)
Using cached lark-1.2.2-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.7/913.7 kB 107.4 kB/s eta 0:00:00a 0:00:02
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)
Note: you may need to restart the kernel to use updated packages.
